In [1]:
!git clone https://github.com/shashnkvats/Indofashionclip.git

Cloning into 'Indofashionclip'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 17 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (17/17), 6.30 KiB | 6.30 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
import os
os.chdir('/content/Indofashionclip')

In [ ]:
!pip install -r requirements.txt

In [3]:
from google.colab import drive
drive.mount("/content/mydrive")

Mounted at /content/mydrive


In [6]:
# Dataset reference: https://www.kaggle.com/datasets/validmodel/indo-fashion-dataset
!unzip -qq '/content/mydrive/MyDrive/Colab Notebooks/COSE474/archive.zip' # Set your path stored in indo fashion dataset

In [ ]:
!pip install tensorflow-gpu==2.8.0

In [7]:
import json
from PIL import Image

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import clip
from transformers import CLIPProcessor, CLIPModel

In [8]:
# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [9]:
# Choose computation device
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load pre-trained CLIP model
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 183MiB/s]


In [10]:
# Inference Section
path = "/content/Indofashionclip/images/test/1.jpeg" # Set your path
image = Image.open(path)

image_input = preprocess(image).unsqueeze(0).to(device)
indo_classes = ['saree', 'blouse', 'dhoti_pants', 'dupattas', 'gowns', 'kurta_men', 'leggings_and_salwars', 'lehenga', 'mojaris_men', 'mojaris_women', 'nehru_jackets', 'palazzos', 'petticoats', 'sherwanis', 'women_kurta']
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in indo_classes]).to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{indo_classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

           saree: 76.27%
         lehenga: 20.53%
        dupattas: 1.53%
leggings_and_salwars: 0.68%
        palazzos: 0.52%
